## DF1 - Daily stock prices
For this first section, we are using Yahoo Finance to make a pandas dataframe with the 25 companies of c25 

In [5]:
#Installing dependencies
!pip install  yfinance
!pip install  pandas
!pip install  requests
!pip install  bs4


import pandas as pd
import yfinance as yf
import requests as re 
from bs4 import BeautifulSoup
 

In [178]:
#Creating a list for the Yahoo ticker names of the OMX stocks 
omx_tickers = ["COLO-B.CO", "CHR.CO", "TRYG.CO", "GN.CO", "ROCK-B.CO", "GMAB.CO", "FLS.CO", "NOVO-B.CO", "MAERSK-B.CO", "DSV.CO", "ISS.CO", "BAVA.CO", "NETC.CO", "MAERSK-A.CO", "CARL-B.CO", "ORSTED.CO", "RBREW.CO", "NZYM-B.CO", "DEMANT.CO", "NDA-DK.CO", "PNDORA.CO", "AMBU-B.CO", "DANSKE.CO", "VWS.CO", "JYSK.CO"]

# Use the yfinance library to retrieve the data for each ticker.
data = yf.download(omx_tickers, start='2010-01-01', end='2022-12-31')

# Create a pandas dataframe from the retrieved data
df_daily = pd.DataFrame(data)


[*********************100%***********************]  25 of 25 completed


### Converting from wide to long format

In [186]:
#Creating two seperate dataframes for sales volume and close price.
df_daily_volume = df_daily['Volume'].reset_index().melt(id_vars=["Date"])
df_daily_close = df_daily['Adj Close'].reset_index().melt(id_vars=["Date"])


#Renaming columns
df_daily_volume = df_daily_volume.rename(columns={"variable": "Ticker", "value": "Volume"})
df_daily_close = df_daily_close.rename(columns={"variable": "Ticker", "value": "Close"}) 

#Joining the two tables
df_daily_combined = pd.merge(df_daily_close, df_daily_volume, on=["Date", "Ticker"])

#Creating YearMo column, to be used as key to inflation df. 
df_daily_combined['Year'] = df_daily_combined['Date'].dt.year.apply(str)
df_daily_combined['Month'] = df_daily_combined['Date'].dt.month.apply(lambda x: str(x).zfill(2)) #lambda used for leading 0
df_daily_combined['YearMo'] = df_daily_combined['Year'] +'-' + df_daily_combined['Month']

df_daily_combined


,Date,Ticker,Close,Volume,Year,Month,YearMo
0,2010-01-04,AMBU-B.CO,5.007598,124420.0,2010,01,2010-01
1,2010-01-05,AMBU-B.CO,4.961657,910560.0,2010,01,2010-01
2,2010-01-06,AMBU-B.CO,5.191363,51220.0,2010,01,2010-01
3,2010-01-07,AMBU-B.CO,5.053539,39060.0,2010,01,2010-01
4,2010-01-08,AMBU-B.CO,5.099481,69160.0,2010,01,2010-01
...,...,...,...,...,...,...,...
81070,2022-12-16,VWS.CO,193.619995,3711356.0,2022,12,2022-12
81071,2022-12-19,VWS.CO,196.839996,1172158.0,2022,12,2022-12
81072,2022-12-20,VWS.CO,191.000000,2206281.0,2022,12,2022-12
81073,2022-12-21,VWS.CO,193.759995,1343059.0,2022,12,2022-12


## Data Cleaning 

In [187]:
# Counting the number of missing values in each column for the long dataframe
null_counts_long = df_daily_combined.isna().sum()
print(null_counts_long)
#This gives 4942 for all continious columns


# Counting the number of missing values in each column for the wide/imported dataframe
null_counts_wide = df_daily[['Adj Close', 'Volume']].isna().sum()
#print(null_counts_wide)
print(sum(null_counts_wide))
#Gets 9884 /2 = 4942, so long == Wide in terms of NA. I see the largest amount of NaN values for Net Company
#Which makes logical sense, as they have not been publicly traded for the whole duration. Furhtermore, volume = adj Close, 
#So it can be assumed that NAN is across all continuous columns (meaning either all are NaN or None are NaN). 
#Deleting missing rows is suitable for the long DF, as records will not be lost for the other stocks, but first I will check if there 
#are any values that are not missing due to the stock not being traded





Date         0
Ticker       0
Close     4942
Volume    4942
Year         0
Month        0
YearMo       0
dtype: int64
9884


### Checking for Missing values that are between two Non-NaN values

In [189]:
# Shifting the "Close" column up and down by one row

df_daily_combined["Close_shift_up"] = df_daily_combined["Close"].shift(1)
df_daily_combined["Close_shift_down"] = df_daily_combined["Close"].shift(-1)

# Creating a boolean mask indicating rows with missing values between non-missing values
mask = df_daily_combined["Close"].isna() & df_daily_combined["Close_shift_up"].notna() & df_daily_combined["Close_shift_down"].notna()

# Using the mask to filter the rows with missing values between non-missing values
df_filtered = df_daily_combined[mask]

# The resulting DataFrame will only include rows with missing values between non-missing values
df_filtered
df_daily_combined[55210:55230]


#Here I find only one missing observation, for Novo Nordisk, which proves a pretty sound dataframe. 
#As only one observation is missing, It will be decided to drop all Na.



,Date,Ticker,Close,Volume,Year,Month,YearMo,Close_shift_up,Close_shift_down
55210,2010-04-29,NOVO-B.CO,70.590820,7569490.0,2010,04,2010-04,68.089485,72.509071
55211,2010-05-03,NOVO-B.CO,72.509071,4708340.0,2010,05,2010-05,70.590820,73.000130
55212,2010-05-04,NOVO-B.CO,73.000130,6101230.0,2010,05,2010-05,72.509071,72.739243
55213,2010-05-05,NOVO-B.CO,72.739243,5750220.0,2010,05,2010-05,73.000130,71.143288
55214,2010-05-06,NOVO-B.CO,71.143288,8669745.0,2010,05,2010-05,72.739243,69.056236
55215,2010-05-07,NOVO-B.CO,69.056236,11640930.0,2010,05,2010-05,71.143288,70.744286
55216,2010-05-10,NOVO-B.CO,70.744286,8071090.0,2010,05,2010-05,69.056236,71.925919
55217,2010-05-11,NOVO-B.CO,71.925919,4003290.0,2010,05,2010-05,70.744286,72.416992
55218,2010-05-12,NOVO-B.CO,72.416992,6215810.0,2010,05,2010-05,71.925919,NaN
55219,2010-05-14,NOVO-B.CO,NaN,NaN,2010,05,2010-05,72.416992,72.892685


In [190]:
#dropping missing values
df_daily_combined = df_daily_combined.dropna().reset_index(drop=True)

### Adjusting for inflation

In [191]:
######Creating a dataframe for yearly inflation


#Importing the danish YoY CPI change dataset

inflation_df = pd.read_csv('DK inflation.csv', delimiter=";")

#Splitting the YearMonth column into two 
inflation_df[['Year','Month']] = inflation_df['TID'].str.split('M',expand=True)

#Convert the inflation column into float, inorder to be used for calculations
inflation_df['INDHOLD'] = inflation_df['INDHOLD'].str.replace(',', '.') .astype(float)
inflation_df = inflation_df.rename(columns={"INDHOLD": "Inflation"})

#Adding a YearMonth column
inflation_df['YearMo'] = inflation_df['Year'] + '-' + inflation_df['Month']
#Selecting only the neccesary columns
inflation_df = inflation_df[['Year','Month','YearMo','Inflation']]

#As the dataframe does not have inflationnumber for december 2022, it will be manually added
inflation_df.loc[251] = ["2022", "12", "2022-12", 9.7]
inflation_df


,Year,Month,YearMo,Inflation
0,2002,01,2002-01,2.5
1,2002,02,2002-02,2.5
2,2002,03,2002-03,2.6
3,2002,04,2002-04,2.6
4,2002,05,2002-05,2.0
...,...,...,...,...
247,2022,08,2022-08,8.9
248,2022,09,2022-09,10.0
249,2022,10,2022-10,10.1
250,2022,11,2022-11,8.9


In [192]:
####Creating an inflation column to the daily prices dataframe

df_daily_combined['Adjusted Price'] = 0

for index, row in df_daily_combined.iterrows():
    # Getting the date and stock price for the current row
    YearMo = row["YearMo"]
    Close = row["Close"]
    
    # Looking up the corresponding inflation rate in the inflation dataset using the YearMo column
    inflation_row = inflation_df[inflation_df["YearMo"] == YearMo]
    #Gets the inflation number
    Inflation = inflation_row["Inflation"].iloc[0]
    # Calculate the inflation-adjusted price for the stock price
    adjusted_price = Close / (1 + (Inflation/100))
    
    # Store the inflation-adjusted price in the new column in the daily stock price dataset
    df_daily_combined.at[index, "Adjusted Price"] = adjusted_price

df_daily_combined




,Date,Ticker,Close,Volume,Year,Month,YearMo,Close_shift_up,Close_shift_down,Adjusted Price
0,2010-01-05,AMBU-B.CO,4.961657,910560.0,2010,01,2010-01,5.007598,5.191363,4.864370
1,2010-01-06,AMBU-B.CO,5.191363,51220.0,2010,01,2010-01,4.961657,5.053539,5.089572
2,2010-01-07,AMBU-B.CO,5.053539,39060.0,2010,01,2010-01,5.191363,5.099481,4.954450
3,2010-01-08,AMBU-B.CO,5.099481,69160.0,2010,01,2010-01,5.053539,5.191363,4.999491
4,2010-01-11,AMBU-B.CO,5.191363,54420.0,2010,01,2010-01,5.099481,5.168393,5.089572
...,...,...,...,...,...,...,...,...,...,...
76114,2022-12-15,VWS.CO,201.649994,1891757.0,2022,12,2022-12,205.949997,193.619995,183.819502
76115,2022-12-16,VWS.CO,193.619995,3711356.0,2022,12,2022-12,201.649994,196.839996,176.499540
76116,2022-12-19,VWS.CO,196.839996,1172158.0,2022,12,2022-12,193.619995,191.000000,179.434819
76117,2022-12-20,VWS.CO,191.000000,2206281.0,2022,12,2022-12,196.839996,193.759995,174.111212


## Adding Daily Technical metrics 

In [195]:
#Adding DMA columsn to the dataframe.

df_daily_combined["DMA30"] = df_daily_combined["Close"].rolling(30).mean()
df_daily_combined["DMA50"] = df_daily_combined["Close"].rolling(50).mean()
df_daily_combined["DMA200"] = df_daily_combined["Close"].rolling(200).mean()



,Date,Ticker,Close,Volume,Year,Month,YearMo,Close_shift_up,Close_shift_down,Adjusted Price,DMA30,DMA50,DMA200
25,2010-02-09,AMBU-B.CO,5.650776,112540.0,2010,02,2010-02,5.742658,5.650776,5.539977,NaN,NaN,NaN
26,2010-02-10,AMBU-B.CO,5.650776,277080.0,2010,02,2010-02,5.650776,5.650776,5.539977,NaN,NaN,NaN
27,2010-02-11,AMBU-B.CO,5.650776,62000.0,2010,02,2010-02,5.650776,5.650776,5.539977,NaN,NaN,NaN
28,2010-02-12,AMBU-B.CO,5.650776,159760.0,2010,02,2010-02,5.650776,5.765629,5.539977,NaN,NaN,NaN
29,2010-02-15,AMBU-B.CO,5.765629,73400.0,2010,02,2010-02,5.650776,5.696717,5.652578,5.447869,NaN,NaN
30,2010-02-16,AMBU-B.CO,5.696717,88160.0,2010,02,2010-02,5.765629,5.696717,5.585017,5.472371,NaN,NaN
31,2010-02-17,AMBU-B.CO,5.696717,98000.0,2010,02,2010-02,5.696717,5.696717,5.585017,5.489216,NaN,NaN
32,2010-02-18,AMBU-B.CO,5.696717,49220.0,2010,02,2010-02,5.696717,5.696717,5.585017,5.510655,NaN,NaN
33,2010-02-19,AMBU-B.CO,5.696717,21500.0,2010,02,2010-02,5.696717,5.650776,5.585017,5.530563,NaN,NaN
34,2010-02-22,AMBU-B.CO,5.650776,23880.0,2010,02,2010-02,5.696717,5.627805,5.539977,5.545877,NaN,NaN
